In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import time
import random

In [ ]:
print("Loading MNIST dataset...")

# Define the path to the dataset
data_path = '../../data/external'

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load the training set
train_set = torchvision.datasets.MNIST(root=data_path, train=True, download=False, transform=transform)

# Load the test set
test_set = torchvision.datasets.MNIST(root=data_path, train=False, download=False, transform=transform)

print("MNIST dataset loaded successfully!")

In [ ]:
def show_image(img, label):
    plt.figure(figsize=(3,3))
    plt.imshow(img.squeeze(), cmap='gray')
    plt.title(f"Label: {label}")
    plt.axis('on')
    plt.show()
    
# Get a random image from the training set
random_idx = torch.randint(0, len(train_set), (1,)).item()
img, label = train_set[random_idx]

print(f"Displaying a random image from MNIST (index: {random_idx})")
show_image(img, label)

In [ ]:
# Print some information about the dataset
print(f"Number of training samples: {len(train_set)}")
print(f"Number of test samples: {len(test_set)}")
print(f"Image shape: {train_set[0][0].shape}")

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
# Instantiate the network
model = Net()

print("Convolutional Neural Network defined:")
print(model)

In [ ]:
# If CUDA is available, move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model moved to device: {device}")

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Loss function: Cross Entropy Loss")
print("Optimizer: Adam with learning rate 0.001")

In [ ]:
# Set hyperparameters
batch_size = 64
epochs = 5

# Create data loaders
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# Lists to store metrics
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    correct = 0
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    
    train_loss /= len(train_loader)
    train_accuracy = 100. * correct / len(train_loader.dataset)
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    return train_loss, train_accuracy

In [ ]:
def test(model, device, test_loader):
    model.eval()
    all_preds = []
    all_targets =  []
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    test_loss /= len(test_loader)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({test_accuracy:.2f}%)\n')
    return test_loss, test_accuracy, np.array(all_preds), np.array(all_targets)

In [ ]:
# Training loop
start_time = time.time()
for epoch in range(1, epochs + 1):
    train_loss, train_accuracy = train(model, device, train_loader, optimizer, epoch)
    test_loss, test_accuracy, _, _ = test(model, device, test_loader)

end_time = time.time()
training_time = end_time - start_time
print(f"Training completed in {training_time:.2f} seconds")

# Plotting the results
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_losses, label='Train')
plt.plot(range(1, epochs + 1), test_losses, label='Test')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Test Loss')

plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), train_accuracies, label='Train')
plt.plot(range(1, epochs + 1), test_accuracies, label='Test')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.title('Training and Test Accuracy')

plt.tight_layout()
plt.show()

In [ ]:
def predict_and_display(model, device, dataset, num_images=5):
    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
    model.eval()

    with torch.no_grad():
        for i in range(num_images):
            idx = random.randint(0, len(dataset) - 1)
            img, true_label = dataset[idx]
            
            # Make prediction
            img = img.unsqueeze(0).to(device)
            output = model(img)
            pred_label = output.argmax(dim=1, keepdim=True).item()
            
            # Display image and prediction
            axes[i].imshow(img.cpu().squeeze(), cmap='gray')
            axes[i].set_title(f'Pred: {pred_label}\nTrue: {true_label}')
            axes[i].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
print("Making predictions for 5 random images from the test set:")
predict_and_display(model, device, test_set)

# Calculate overall accuracy on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = 100 * correct / total
print(f'Overall accuracy on the test set: {accuracy:.2f}%')

In [ ]:
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    
    plt.title('Confusion Matrix for MNIST Classification')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

    plt.savefig(f"confusion_matrix.png", dpi=600, bbox_inches='tight', format='png')
    plt.show()

_, _, predictions, targets = test(model, device, test_loader)
plot_confusion_matrix(targets, predictions)

In [ ]:
torch.save(model.state_dict(), '../../model/mnist_cnn.pth')